In [1]:
from IPython.core.debugger import set_trace

In [2]:
import skvideo.io as sio
import skvideo.datasets as datasets
import skvideo.measure as measure

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [46]:
filename = "goldeneye.mp4"
# filename = datasets.bikes()
# data = sio.vread(filename)
metadata = sio.ffprobe(filename)
fps = metadata['video']['@avg_frame_rate']
ffps = [ float(x) for x in fps.split('/') ]
ffps = ffps[0] / ffps[1]
frames = np.int(metadata['video']['@nb_frames'])
width = np.int(metadata['video']['@width'])
height = np.int(metadata['video']['@height'])

In [ ]:
def batcher(iterable, width=5, step=1):
    offset = 0
    batch = []
    while len(batch) < width:
        try:
            el = next(iterable)
        except StopIteration:
            

In [51]:
data = sio.vreader(filename)
counter = 0
acc = []
scene_cuts = [0]
for frame in data:
    acc.append(frame)
    if len(acc) >= ffps:
        batch_cuts = measure.scenedet(acc, method='histogram', parameter1=1.5)[1:] # Don't count first cut (always true)
        if len(batch_cuts) > 0:
            batch_cuts = np.add(batch_cuts, counter).tolist()
            print("Found scene cut in frames:", batch_cuts)
            scene_cuts += batch_cuts
        counter += len(acc) - 2
        acc = acc[-2:]
print(counter, frames, len(acc))
if counter < frames:
    batch_cuts = measure.scenedet(acc, method='histogram', parameter1=1.5)[1:] # Don't count first cut (always true)
    if len(batch_cuts) > 0:
        batch_cuts = np.add(batch_cuts, counter).tolist()
        print("Found scene cut in frames:", batch_cuts)
        scene_cuts += batch_cuts
print(scene_cuts)

Found scene cut in frames: [210]
Found scene cut in frames: [259]
Found scene cut in frames: [374]
Found scene cut in frames: [650]
Found scene cut in frames: [686, 690, 694]
Found scene cut in frames: [818]
Found scene cut in frames: [876]
Found scene cut in frames: [972, 976, 980]
Found scene cut in frames: [1019]
Found scene cut in frames: [1055]
Found scene cut in frames: [1099]
Found scene cut in frames: [1174, 1178]
Found scene cut in frames: [1226]
Found scene cut in frames: [1260]
Found scene cut in frames: [1281]
Found scene cut in frames: [1334]
Found scene cut in frames: [1365]


KeyboardInterrupt: 

In [3]:
def getPlot(edgelist1, t, w, h, T):
    myDPI = 100.0
    fig = plt.figure(figsize=(w/myDPI, h/myDPI), dpi=myDPI)
    plt.title("histogram algorithm")
    plt.plot(edgelist1)
    plt.plot([t, t], [0, 1])
    plt.xlim([0, T])
    plt.ylim([0, 1])
    
    plt.tight_layout()
    fig.canvas.draw()

    # Get the RGBA buffer from the figure
    w,h = fig.canvas.get_width_height()
    buf = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
    buf.shape = (h, w, 4)
    plot = buf[:, :, 1:]
    plt.close()

    return plot

In [37]:
scene_histo_idx = measure.scenedet(sio.vread(filename), method='histogram')
scene_histo = np.zeros((frames,))
scene_histo[scene_histo_idx] = 1

[  0  30  76 137 187 242]


In [6]:
writer = sio.FFmpegWriter("scene_cuts.mp4", inputdict={"-r": fps})

for i in range(frames):
    out = np.zeros((height, width*2, 3))
    chart = getPlot(scene_histo, i, width, height, frames)
    out[:, :width] = data[i]
    out[:, width:] = chart
    writer.writeFrame(out)
    
writer.close()

### Detected cuts
---
<video controls width="100%" src="scene_cuts.mp4" />